In [1]:
import pipeline_funcs as fn
import pandas as pd 
import numpy as np
import pickle

import time

import utils.db_dedup as db_dedup

In [ ]:
gemini_success_path = '../output/subject_line/gemini_success.parquet'
gemini_error_path = '../output/subject_line/gemini_error.parquet'
tagged_sl_path = '../output/subject_line/tagged_subject_line.parquet'
tagged_raw_sl_path = '../output/subject_line/tagged_subject_line_raw.parquet'

gemini_model = 'models/gemini-1.5-flash-latest'
gemini_api_key_path = '../../../docs/api_key/xxx.json'
service_account_json = '../../../docs/api_key/xxx.json'

In [3]:
model = fn.config_gemini(gemini_api_key_path, gemini_model)

In [4]:
df_subject_line_bq = fn.get_sl_from_bq(service_account_json)

In [5]:
fn.generate_responses_gemini(df_subject_line_bq, model, gemini_success_path, gemini_error_path)

  0%|          | 0/223 [00:00<?, ?it/s]

 13%|█▎        | 30/223 [02:36<16:14,  5.05s/it]

Retried 1 times forUpgrade ke📱yang pasti aman!


 20%|██        | 45/223 [04:03<17:11,  5.79s/it]

Retried 1 times forเลือกพับที่ใช่✅โดนใจทุกไลฟ์สไตล์💘


...

Retried 1 times forWant to make the most of your Galaxy?


 68%|██████▊   | 151/223 [14:02<06:12,  5.18s/it]

Retried 1 times for98" Samsung TV comes with travel voucher 🤩


100%|██████████| 223/223 [20:15<00:00,  5.45s/it]


""


In [6]:
fn.sl_tagging(gemini_success_path, tagged_sl_path, tagged_raw_sl_path)

In [7]:
fn.upload_bq(tagged_sl_path, tagged_raw_sl_path, service_account_json)

c:\Users\huydao\AppData\Local\anaconda3\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: length.
  warnings.warn(


In [ ]:
# df_tagged, df_raw = pd.read_parquet('../data/bestpratice/subject_line.parquet'), pd.read_parquet('../data/bestpratice/subject_line_raw.parquet')

# df_tagged = df_tagged.drop_duplicates(subset=['subject_line'])
# df_raw = df_raw.drop_duplicates(subset=['subject_line'])

# df = df_tagged.merge(df_raw, on='subject_line', how='inner')
# df.to_parquet('temp.parquet')

# fn.sl_tagging('temp.parquet', tagged_sl_path, tagged_raw_sl_path)

In [ ]:
# df_tagged[df_tagged.subject_line.duplicated(keep=False)]

df = pd.read_parquet('uniqued.parquet')

lst = []
for start in range(500,800,100):
    print(start)
    if start == 700:
        lst.append(df.iloc[start:,:])
    else:
        lst.append(df.iloc[start:start+100,:])

with open('my_list.pkl', 'rb') as f:
    reopened_lst = pickle.load(f)

for i, chunk in enumerate(reopened_lst):
  print(chunk)
  try:
    fn.generate_responses_gemini(chunk, model, gemini_success_path, gemini_error_path)
    time.sleep(20)  
    fn.sl_tagging(gemini_success_path, tagged_sl_path, tagged_raw_sl_path)
    time.sleep(20)  
    fn.upload_bq(tagged_sl_path, tagged_raw_sl_path, service_account_json)
    time.sleep(50)  
   
  except Exception as e:
    print("Error processing chunk {i}: {e}")


df = pd.read_parquet('uniqued.parquet')

lst = []
for start in range(100,800,100):
    print(start)
    if start == 700:
        lst.append(df.iloc[start:,:])
    else:
        lst.append(df.iloc[start:start+100,:])

with open('my_list.pkl', 'wb') as f:
    pickle.dump(lst, f)


,subject_line,curiosity,urgency,value,relevance,emotion,specificity,product_features
5,We're counting down...,3,2,0,0,2,1,None
11,We're counting down...,4,3,0,0,2,1,
190,Up to RM400* e-vouchers for grabs. T&Cs apply.,4,3,3,0,2,2,
195,Up to RM400* e-vouchers for grabs. T&Cs apply.,4,3,4,0,2,2,
217,It takes less than 5 mins!,3,2,0,0,1,1,
226,It takes less than 5 mins!,4,3,0,0,2,1,
